In [1]:
import os
from sqlalchemy import create_engine
from sqlalchemy import text

# SQLAlchemy - Core
## Engine - a gateway to database connectivity, which provides:
- Connection - the interface to a database connection, which provides:
- Result - the interface to a database result

## Engine maintans behind the screne:
- Dialect - transaltes SLQAlchemy constructs for a specific kind of database and database driver
- Connection Pool - holds a collection of database connections in memory for fast re-use

## SQL Expression Language - Python constructs that represent SQL statements

## Schema/Types - Python constructs that represent tables, columns, datatypes and other DDL

In [2]:
# create_engine() builds a *factory* for database connections.
# Below we create an engine that will connect to a SQLite database.
# "future" means we want the full 2.0 behavior.
if os.path.exists("database.db"):
    os.remove("database.db")
engine = create_engine("sqlite:///database.db", future=True)

## Differrence between using engine.begin() and engine.connect()
### **`Begin()`** auto-commits, while you have to commit yourself if you use **`connect()`**


    with engine.connect() as conn:
        with conn.begin():
            conn.execute(...)
            conn.execute(...)


    with engine.connect() as conn:
        conn.execute(...)
        conn.execute(...)
        conn.commit()

It is now common to use with statements when executing code. If we did not use with statements we would have to remember to use **`connection.close()`**

In [3]:
with engine.begin() as conn:
    conn.execute(
        text(
            """
        create table employee (
            emp_id integer primary key,
            emp_name varchar
        )
    """
        )
    )

    conn.execute(
        text(
            """
        create table employee_of_month (
            emp_id integer primary key,
            emp_name varchar
        )
    """
        )
    )

    conn.execute(
        text("insert into employee(emp_name) values (:name)"),
        [{"name": "spongebob"}, {"name": "sandy"}, {"name": "squidward"}],
    )

In [4]:
with engine.connect() as connection:
    rows = connection.execute(text("select * from employee")).all()
    for row in rows:
        print(row)

(1, 'spongebob')
(2, 'sandy')
(3, 'squidward')


In [10]:
with engine.connect() as connection:
    result = connection.execute(text('select * from employee')).all()
    for emp_id, emp_name in result:
        print(f'emp_id: {emp_id}, emp_name: {emp_name}')

emp_id: 1, emp_name: spongebob
emp_id: 2, emp_name: sandy
emp_id: 3, emp_name: squidward


In [6]:
# The Connection then features an .execute() method that will run queries.
# To invoke a textual query we use the text() construct
with engine.connect() as connection:
    stmt = text("select emp_id, emp_name from employee where emp_id=:emp_id")
    result = connection.execute(stmt, {"emp_id": 2})
    # the result object we get back is similar to a cursor, with more methods,
    # such as first() which will return the first row and close the result set
    row = result.first()
    print(row.emp_name)

sandy


In [7]:
# Unlike previous SQLAlchemy versions, SQLAlchemy 2.0 has no concept
# of "library level" autocommit; which means, if the DBAPI driver is in
# a transaction, SQLAlchemy will never commit it automatically.   The usual
# way to commit is called "commit as you go"

with engine.connect() as connection:
    connection.execute(
        text("insert into employee_of_month (emp_name) values (:emp_name)"),
        {"emp_name": "sandy"},
    )
    connection.commit()

In [8]:
# the other way is called "begin once", when you just have a single transaction
# to commit

with engine.begin() as connection:
    connection.execute(
        text("insert into employee_of_month (emp_name) values (:emp_name)"),
        {"emp_name": "squidward"},
    )

In [11]:
# You can also use begin() blocks local to the connection

with engine.connect() as connection:
    with connection.begin():
        connection.execute(
            text("update employee_of_month set emp_name = :emp_name"),
            {"emp_name": "squidward"},
        )
    # end of inner block: commits transaction, or rollback if exception
 # end of outer block: releases connection to the connection pool

In [12]:
# transactions support "nesting", which is implemented using the
# SAVEPOINT construct.

with engine.connect() as connection:
    with connection.begin():
        savepoint = connection.begin_nested()
        connection.execute(
            text("update employee_of_month set emp_name = :emp_name"),
            {"emp_name": "patrick"},
        )
        savepoint.rollback()  # sorry patrick

        with connection.begin_nested() as savepoint:
            connection.execute(
                text("update employee_of_month set emp_name = :emp_name"),
                {"emp_name": "spongebob"},
            )
            # releases savepoint

        # commits transaction, or rollback if exception
    # releases connection to the connection pool

In [13]:
# most DBAPIs support autocommit now, which is why SQLAlchemy no longer
# does.  To use driver level autocommit, use execution options:

with engine.connect().execution_options(isolation_level="AUTOCOMMIT") as connection:
    connection.execute(
        text("insert into employee(emp_name) values (:name)"),
        {"name": "plankton"},
    )

In [14]:
# the data was autocommitted
with engine.connect() as connection:
    planktons_id = connection.execute(
        text("select emp_id from employee where emp_name=:name"),
        {"name": "plankton"}
    ).scalar()
    print(planktons_id)

4
